# Understanding US Immigration
### Data Engineering Capstone Project

#### Project Summary

This project processes raw US immigration data into a analytics-ready data warehouse, supported by city and airport dimensions. Spark is used to process large volumes of data, and S3 is used for storage to provide visibility into the pipeline and results.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
import utils as uz

### Step 1: Scope the Project and Gather Data & Step 2: Explore and Assess the Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

### Airport Data Overview

- This is a simple table of airport codes and corresponding cities. It comes from here: https://datahub.io/core/airport-codes#data. 
- About 55k airports represented from around the world
- `coordinates` are a string representation of latitude and longitude
- Few missing values / pretty good data quality, other than the `_code` columns and `elevation_ft`

In [3]:
# Read in the airport data
io = uz.S3IO('airport', 'raw')
airport_raw = io.get()
uz.df_overview(airport_raw)

Getting: raw - airport
root
 |-- ident: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- elevation_ft: integer (nullable = true)
 |-- continent: string (nullable = true)
 |-- iso_country: string (nullable = true)
 |-- iso_region: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- gps_code: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- local_code: string (nullable = true)
 |-- coordinates: string (nullable = true)

Shape: (55075, 12)
+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|ident|         type|                name|elevation_ft|continent|iso_country|iso_region|municipality|gps_code|iata_code|local_code|         coordinates|
+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|  

#### Airport Data Exploratory Analysis

- Airport `type` is the type of airport category, most being `small_airport` and some `closed` / inactive
- All continents are covered

In [5]:
uz.value_counts(airport_raw, 'type')
uz.value_counts(airport_raw, 'continent')
uz.value_counts(airport_raw, 'iso_country')
airport_raw.describe(['elevation_ft']).show()

type
+--------------+-----+
|          type|count|
+--------------+-----+
| small_airport|33965|
|      heliport|11287|
|medium_airport| 4550|
|        closed| 3606|
| seaplane_base| 1016|
| large_airport|  627|
|   balloonport|   24|
+--------------+-----+

continent
+---------+-----+
|continent|count|
+---------+-----+
|       NA|27719|
|       EU| 7840|
|       SA| 7709|
|       AS| 5350|
|       AF| 3362|
|       OC| 3067|
|       AN|   28|
+---------+-----+

iso_country
+-----------+-----+
|iso_country|count|
+-----------+-----+
|         US|22757|
|         BR| 4334|
|         CA| 2784|
|         AU| 1963|
|         KR| 1376|
|         MX| 1181|
|         RU| 1040|
|         DE|  947|
|         GB|  911|
|         FR|  850|
|         AR|  848|
|         CO|  706|
|         IT|  671|
|         PG|  593|
|         VE|  592|
|         ZA|  489|
|         CL|  474|
|         ID|  470|
|         ES|  416|
|         CN|  404|
+-----------+-----+
only showing top 20 rows

+-------+-----

### City Data Overview

- U.S. City Demographic Data: This data comes from OpenSoft. You can read more about it here: https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/.
- About 3K US cities with very few missing data points
- A snapshot of demographics data, last modified in 2018
- Some columns are being read in as continuous numbers, rather than integers
- Unsure what `Count` means

In [6]:
io = uz.S3IO('city', 'raw')
city_raw = io.get()
uz.df_overview(city_raw)

Getting: raw - city
root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Median Age: double (nullable = true)
 |-- Male Population: integer (nullable = true)
 |-- Female Population: integer (nullable = true)
 |-- Total Population: integer (nullable = true)
 |-- Number of Veterans: integer (nullable = true)
 |-- Foreign-born: integer (nullable = true)
 |-- Average Household Size: double (nullable = true)
 |-- State Code: string (nullable = true)
 |-- Race: string (nullable = true)
 |-- Count: integer (nullable = true)

Shape: (2891, 12)
+----------------+-------------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+--------------------+-----+
|            City|        State|Median Age|Male Population|Female Population|Total Population|Number of Veterans|Foreign-born|Average Household Size|State Code|                Race|Count|
+----------------+-------------+----------+---------

#### City Data Exploratory Analysis

- Rows are repeated for each different race. This could be improved upon.
- There appear to be 5 different races represented (not too many to put in separate columns)

In [9]:
city_raw.where(uz.F.col("City") == "Silver Spring").where(uz.F.col("City") == "Silver Spring").show()

+-------------+--------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+--------------------+-----+
|         City|   State|Median Age|Male Population|Female Population|Total Population|Number of Veterans|Foreign-born|Average Household Size|State Code|                Race|Count|
+-------------+--------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+--------------------+-----+
|Silver Spring|Maryland|      33.8|          40601|            41862|           82463|              1562|       30908|                   2.6|        MD|  Hispanic or Latino|25924|
|Silver Spring|Maryland|      33.8|          40601|            41862|           82463|              1562|       30908|                   2.6|        MD|               White|37756|
|Silver Spring|Maryland|      33.8|          40601|            41862|           82463|              

In [12]:
uz.value_counts(city_raw, 'State')
uz.value_counts(city_raw, 'Race')
city_raw.describe(['Median Age', 'Average Household Size']).show()

State
+--------------+-----+
|         State|count|
+--------------+-----+
|    California|  676|
|         Texas|  273|
|       Florida|  222|
|      Illinois|   91|
|    Washington|   85|
|       Arizona|   80|
|      Colorado|   80|
|      Michigan|   79|
|North Carolina|   70|
|      Virginia|   70|
| Massachusetts|   69|
|    New Jersey|   57|
|       Georgia|   55|
|     Minnesota|   54|
|      New York|   54|
|       Indiana|   51|
|      Maryland|   50|
|          Ohio|   49|
|          Utah|   48|
|     Wisconsin|   45|
+--------------+-----+
only showing top 20 rows

Race
+--------------------+-----+
|                Race|count|
+--------------------+-----+
|  Hispanic or Latino|  596|
|               White|  589|
|Black or African-...|  584|
|               Asian|  583|
|American Indian a...|  539|
+--------------------+-----+

+-------+-----------------+----------------------+
|summary|       Median Age|Average Household Size|
+-------+-----------------+--------------------

### SAS Data Overview

- date columns are being read in as floats
- lots of data that isn't human readable / needs to be mapped
- also, some text-based columns could be mapped in a transform step for storage optimization
- columns need to be renamed for readability
- `I94PORT`, `I94CIT`, `I94RES` need cleaning to derive city|state|country info as relevant

In [13]:
io = uz.S3IO('sas', 'raw')
sas_raw = io.get()
uz.df_overview(sas_raw)

Getting: raw - sas
root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: 

#### SAS Data Exploratory Analysis

- Looks like all data is from April of 2016
- pretty evenly-spread birth years

In [16]:
sas_raw.select('i94yr','i94mon').distinct().show()

+------+------+
| i94yr|i94mon|
+------+------+
|2016.0|   4.0|
+------+------+



In [18]:
uz.value_counts(sas_raw, 'gender')
uz.value_counts(sas_raw, 'matflag')
uz.value_counts(sas_raw, 'biryear')

gender
+------+-------+
|gender|  count|
+------+-------+
|     M|1377224|
|     F|1302743|
|  null| 414269|
|     X|   1610|
|     U|    467|
+------+-------+

matflag
+-------+-------+
|matflag|  count|
+-------+-------+
|      M|2957884|
|   null| 138429|
+-------+-------+

biryear
+-------+-----+
|biryear|count|
+-------+-----+
| 1986.0|71958|
| 1983.0|70415|
| 1985.0|70409|
| 1982.0|70251|
| 1984.0|69809|
| 1981.0|69626|
| 1980.0|67960|
| 1987.0|67762|
| 1976.0|66568|
| 1979.0|66494|
| 1988.0|65566|
| 1978.0|64262|
| 1977.0|63035|
| 1975.0|62622|
| 1974.0|62150|
| 1971.0|62075|
| 1972.0|62001|
| 1973.0|61430|
| 1989.0|60340|
| 1970.0|59730|
+-------+-----+
only showing top 20 rows



In [4]:
io = uz.S3IO('sas', 'staging')
min_date = io.get().agg({"arrdate": "min"}).collect()[0]['min(arrdate)']
print(f"Earliest date available in SAS data: {min_date}")

Getting: staging - sas
Earliest date available in SAS data: 2016-04-01


### Weather Data Overview

- This dataset contains temperature information from around the world in various groupings - city, state, etc.
- As you can see from the analysis section, the _dates don't overlap with the SAS data, and thus, we cannot use this dataset_

In [8]:
io = uz.S3IO('GlobalTemperatures', 'raw') # for debugging
weather = io.get().where(uz.F.col("LandAverageTemperature").isNotNull())
uz.df_overview(weather)

root
 |-- dt: date (nullable = true)
 |-- LandAverageTemperature: double (nullable = true)
 |-- LandAverageTemperatureUncertainty: double (nullable = true)
 |-- LandMaxTemperature: double (nullable = true)
 |-- LandMaxTemperatureUncertainty: double (nullable = true)
 |-- LandMinTemperature: double (nullable = true)
 |-- LandMinTemperatureUncertainty: double (nullable = true)
 |-- LandAndOceanAverageTemperature: double (nullable = true)
 |-- LandAndOceanAverageTemperatureUncertainty: double (nullable = true)

Shape: (3180, 9)
+----------+----------------------+---------------------------------+------------------+-----------------------------+------------------+-----------------------------+------------------------------+-----------------------------------------+
|        dt|LandAverageTemperature|LandAverageTemperatureUncertainty|LandMaxTemperature|LandMaxTemperatureUncertainty|LandMinTemperature|LandMinTemperatureUncertainty|LandAndOceanAverageTemperature|LandAndOceanAverageTemperature

### Weather Data Exploratory Analysis

- The maximum date represented in the weather dataset(s) is 12/1/2015 and the minimum date represented in the SAS dataset is 4/1/2016
- Therefore, it would be fruitless to join weather data to this data model

In [6]:
weather = weather.withColumn("dt", uz.F.col('dt').cast("date"))
min_date = uz.datetime.date(2016, 4, 1) # from previous SAS cell
assert weather.groupby('dt').count().agg({'count': 'max'}).collect()[0]['max(count)'] == 1, "Possible duplicate dates. Should have 1 row per date"

Getting: raw - globaltemperatures


In [9]:
print("The data range for GlobalTemperatures is:")
weather.agg({"dt": "max"}).collect()

The data range for GlobalTemperatures is:


[Row(max(dt)=datetime.date(2015, 12, 1))]

- As you can see, no weather datasets overlap dates with the SAS data...

In [2]:
for key in ['GlobalLandTemperaturesByCity', 'GlobalLandTemperaturesByMajorCity','GlobalLandTemperaturesByCountry','GlobalLandTemperaturesByState','GlobalTemperatures']:
    io = uz.S3IO(key,'raw' )
    df = io.get()
    if key == 'GlobalTemperatures':
        df = df.where(uz.F.col("LandAverageTemperature").isNotNull())
    else:
        df = df.where(uz.F.col("AverageTemperature").isNotNull())\
            .filter("Country = 'United States'")
    res = df.agg({"dt": "max"}).collect()[0]['max(dt)']
    print(key, 'max available date:', res)

Getting: raw - globallandtemperaturesbycity
GlobalLandTemperaturesByCity max available date: 2013-09-01 00:00:00
Getting: raw - globallandtemperaturesbymajorcity
GlobalLandTemperaturesByMajorCity max available date: 2013-09-01 00:00:00
Getting: raw - globallandtemperaturesbycountry
GlobalLandTemperaturesByCountry max available date: 2013-09-01 00:00:00
Getting: raw - globallandtemperaturesbystate
GlobalLandTemperaturesByState max available date: 2013-09-01 00:00:00
Getting: raw - globaltemperatures
GlobalTemperatures max available date: 2015-12-01 00:00:00


In [ ]:
# save raw files to S3 bucket
# NOTE: this was done in a console outside the purview of this notebook
uz.load_raw_files_to_s3()

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

##### Airport Cleaning Steps

###### Airport To Staging

- separate `latitude` and `longitude` columns, and cast as doubles
- strip text fields - `iso_region`, `municipality`
- filter to USA only
- derive `state` field from iso_region

###### Airport To Warehouse

- append additional cities from `airport` to `city`
- add `city_id` from city to airport
- create `id` column

##### Cities Cleaning Steps

###### Cities to Staging

- rename various columns
- select distinct rows, ignoring race
- append race column counts for each race, respectively

###### Cities to Warehouse

- append additional locations from `sas` and `airport` tables
- create `id` column

##### SAS Cleaning Steps

###### SAS to Staging

- rename various columns
    `I94YR` to `year`
    `I94MON` to `month`
    `I94CIT` to `I94CIT`
    `I94RES` to `I94RES`
    `I94PORT` to `port`
    `ARRDATE` to `arrdate`
    `I94MODE` to `arrmode`
    `I94ADDR` to `addr_state`
    `DEPDATE` to `depdate`
    `I94BIR` to `age`
    `I94VISA` to `visa`
    `MATFLAG` to `match_flag`
    `BIRYEAR` to `birth_year`
    `GENDER` to `gender`
    `AIRLINE` to `airline`
    `I94YR` to `year`
    `admnum` to `admission_num`
    `FLTNO` to `flight_num`
- cast columns read in as floats into integers (`birth_year`, `year`, `month`, `age`, `arrmode`, `I94CIT`, `I94RES`, `arrdate`, `depdate`, `visa`, `cicid`)
- Apply full-text maps from dataset to create `*_id` columns and drop the full-text columns for reduced storage requirements and save to `SAS_ID_TO_FULLTEXT_MAPS_FPATH`
- convert `match_flag` to boolean
- convert `arrdate` and `depdate` to `Date` type

###### SAS to Warehouse

- append additional locations to `city` table
- add `city_id` column
- add `id` column

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

- Normalize SAS dataset because it is the fact table in this scenario, which saves storage space and enhances performance
- Consider the `airport` and `city` tables as dimensions
- Apply mapping on read from data warehouse for human-readable filtering for mostly `sas`, but also, some for `airport`
- User can quickly merge datasets with `city_id` column

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

- Save raw files to S3 bucket
- Transform `sas`, `airport`, `city` to staging (store in S3 or transform on the fly)
- Transform `sas`, `airport`, `city` to warehouse (store in S3 or transform on the fly)
- Apply various mappings upon read from warehouse

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

### Transform the datasets TO warehouse

- Transforms the data from staging to storage in the data warehouse
- Data is cleaned and formatted to be what it will look like when it is stored in the data warehouse
- Because I was having internet speed issues (probably due to COV19), this data was not stored in the requisite S3 bucket, and instead is displayed here for demonstration purposes

In [2]:
trans = uz.Transformer(None, 'warehouse', write=False, verbose=True)
trans.transform()

Preparing all datasets for storage in the data warehouse
Transforming city to staging as an intermediary step
Loading raw city file from Udacity workspace
Getting: raw - city
Transforming airport to staging as an intermediary step
Getting: raw - airport
Transforming sas to staging as an intermediary step
Getting: raw - sas
Creating id columns from full-text columns for faster querying...
...creating visatype_id to visatype mapping
...creating airline_id to airline mapping
...creating gender_id to gender mapping
Creating city_id columns...
...get cities not represented in cities dimension from SAS and append to city table
......sas_unique_locations shape: (280, 2)
......Additional locations from SAS: 205
	city table shape: (801, 15)
...get cities not represented in cities dimension from AIRPORT and append to city table
......airport_unique_locations shape: (11951, 2)
......Additional locations from AIRPORT: 11388
	city table shape: (12189, 15)
Add monotonically increasing city ID to cit

### GET the datasets FROM warehouse

- Because the transformed data was never saved to S3 (due to internet connectivity issues, described above), the entire data pipeline is recreated here
- In practice, normalized tables would _likely_ be queried directly from normalized tables in S3 for performance

In [9]:
dwh = uz.S3IO(None, 'warehouse', from_raw=True)
dwh.get()

Getting: warehouse - None
Preparing all datasets for storage in the data warehouse
Transforming city to staging as an intermediary step
Loading raw city file from Udacity workspace
Getting: raw - city
Transforming airport to staging as an intermediary step
Getting: raw - airport
Transforming sas to staging as an intermediary step
Getting: raw - sas
Creating id columns from full-text columns for faster querying...
...creating visatype_id to visatype mapping
...creating airline_id to airline mapping
...creating gender_id to gender mapping
Creating city_id columns...
...get cities not represented in cities dimension from SAS and append to city table
......sas_unique_locations shape: (280, 2)
......Additional locations from SAS: 205
	city table shape: (801, 15)
...get cities not represented in cities dimension from AIRPORT and append to city table
......airport_unique_locations shape: (11951, 2)
......Additional locations from AIRPORT: 11388
	city table shape: (12189, 15)
Add monotonically

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [27]:
uz.check_has_data(dwh.sas, 'sas')
uz.check_no_empty_columns(dwh.sas, 'sas')
uz.check_has_data(dwh.city, 'city')
uz.check_no_empty_columns(dwh.city, 'city')
uz.check_has_data(dwh.airport, 'airport')
uz.check_no_empty_columns(dwh.airport, 'airport')

sas has data!
All columns in sas have data
city has data!
All columns in city have data
airport has data!
All columns in airport have data


#### Data Views from DWH

##### SAS DWH Data View

In [23]:
uz.df_overview(dwh.sas, show_nulls=False)

root
 |-- cicid: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- I94CIT: integer (nullable = true)
 |-- I94RES: integer (nullable = true)
 |-- port: string (nullable = true)
 |-- arrdate: date (nullable = true)
 |-- arrmode_id: integer (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- depdate: date (nullable = true)
 |-- age: integer (nullable = true)
 |-- visa_id: integer (nullable = true)
 |-- match_flag: boolean (nullable = false)
 |-- birth_year: integer (nullable = true)
 |-- insnum: string (nullable = true)
 |-- admission_num: double (nullable = true)
 |-- flight_num: string (nullable = true)
 |-- visatype_id: long (nullable = true)
 |-- airline_id: long (nullable = true)
 |-- gender_id: long (nullable = true)
 |-- id: long (nullable = false)
 |-- city_id: long (nullable = true)
 |-- arrmode: string (nullable = true)
 |-- visa: string (nullable = true)
 |-- cit_country: string (nullable = true)
 |-- res_coun

##### City DWH Data View

In [24]:
uz.df_overview(dwh.city, show_nulls=False)

root
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- state_full: string (nullable = true)
 |-- median_age: double (nullable = true)
 |-- male_pop: integer (nullable = true)
 |-- female_pop: integer (nullable = true)
 |-- total_pop: integer (nullable = true)
 |-- num_veterans: integer (nullable = true)
 |-- foreign_born: integer (nullable = true)
 |-- avg_household_size: double (nullable = true)
 |-- Black or African-American: integer (nullable = true)
 |-- Hispanic or Latino: integer (nullable = true)
 |-- White: integer (nullable = true)
 |-- Asian: integer (nullable = true)
 |-- American Indian and Alaska Native: integer (nullable = true)
 |-- id: long (nullable = false)

Shape: (12189, 16)
+-----------+-----+--------------+----------+--------+----------+---------+------------+------------+------------------+-------------------------+------------------+------+-----+---------------------------------+---+
|       city|state|    state_full|median_age|male_p

##### Airport DWH Data View

In [25]:
uz.df_overview(dwh.airport, show_nulls=False)

root
 |-- ident: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- elevation_ft: integer (nullable = true)
 |-- continent: string (nullable = true)
 |-- iso_country: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- id: long (nullable = false)
 |-- city_id: long (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)

Shape: (22757, 12)
+-----+-------------+--------------------+------------+---------+-----------+------------------+------------------+-----+-----------+-------------+-----+
|ident|         type|                name|elevation_ft|continent|iso_country|          latitude|         longitude|   id|    city_id|         city|state|
+-----+-------------+--------------------+------------+---------+-----------+------------------+------------------+-----+-----------+-------------+-----+
| 58FD|small_airport|   Southerly Airport|         

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### SAS Data Dictionary

- `id`: long (nullable = false)
- `cicid`: integer (nullable = true)
- `year`: integer (nullable = true)
  - Year of immigration status change
- `month`: integer (nullable = true)
  - Month of immigration status change
- `I94CIT`: integer (nullable = true)
  - Country of citizenship code
- `I94RES`: integer (nullable = true)
  - Country of residence code
- `cit_country`: string (nullable = true)
  - Country of citizenship full text
- `res_country`: string (nullable = true)
  - Country of residence full text
- `port`: string (nullable = true)
  - Port of entry
- `arrdate`: date (nullable = true)
  - Arrival date
- `arrmode`: string (nullable = true)
  - Mode of arrival (Land, Sea, Air)
- `arrmode_id`: integer (nullable = true)
  - `arrmode` ID
- `addr_state`: string (nullable = true)
  - State in which they live
- `depdate`: date (nullable = true)
  - Departure date
- `age`: integer (nullable = true)
  - Age at the time of the record
- `birth_year`: integer (nullable = true)
  - Year in which person was born
- `visa`: string (nullable = true)
  - Visa category - Business, Pleasure, Student
- `visa_id`: integer (nullable = true)
  - `visa` normalized
- `match_flag`: boolean (nullable = false)
  - Match flag - Match of arrival and departure records
- `insnum`: string (nullable = true)
  - INS number
- `admission_num`: double (nullable = true)
  - Admission number 
- `flight_num`: string (nullable = true)
  - Flight number
- `visatype`: string (nullable = true)
  - Class of admission legally admitting the non-immigrant to temporarily stay in U.S.
- `visatype_id`: long (nullable = true)
  - Visatype normalized
- `airline`: string (nullable = true)
  - Airline string code abbreviation
- `airline_id`: long (nullable = true)
  - `airline` normalized
- `gender`: string (nullable = true)
  - Gender string code
- `gender_id`: long (nullable = true)
  - `gender` normalized
- `city_id`: long (nullable = true)
  - Port city id, joinable on `city.id`
- `city`: string (nullable = true)
  - Port city full text
- `state`: string (nullable = true)
  - Port city full text

#### City Data Dictionary

- `id`: long (nullable = false)
- `city`: string (nullable = true)
- `state`: string (nullable = true)
  - State abbreviation
- `state_full`: string (nullable = true)
  - State spelled out
- `median_age`: double (nullable = true)
  - Median age of all residents
- `male_pop`: integer (nullable = true)
  - Count of males
- `female_pop`: integer (nullable = true)
  - Count of females
- `total_pop`: integer (nullable = true)
  - Count of all residents
- `num_veterans`: integer (nullable = true)
  - Count of veterans
- `foreign_born`: integer (nullable = true)
  - Count of foreign-born citizens
- `avg_household_size`: double (nullable = true)
  - Average number of people in a house
- `Black or African-American`: integer (nullable = true)
  - Count of black/African-Americans
- `Hispanic or Latino: integer` (nullable = true)
  - Count of hispanic/latinos
- `White`: integer (nullable = true)
  - Count of white people
- `Asian`: integer (nullable = true)
  - Count of asians
- `American Indian and Alaska Native`: integer (nullable = true)
  - Count of Native Americans

#### Airport Data Dictionary

- `id`: long (nullable = false)
  - Unique row identifier
- `ident`: string (nullable = true)
  - Identifier code
- `type`: string (nullable = true)
  - Airport type (small_airport, heliport, medium_airport, closed, seaplane_base, large_airport, balloonport)
- `name`: string (nullable = true)
  - Name of airport
- `elevation_ft`: integer (nullable = true)
  - Elevation Feet
- `continent`: string (nullable = true)
  - Continent of airport location (filtered to be only NA)
- `iso_country`: string (nullable = true)
  - Country of location (filtered to be only US)
- `latitude`: double (nullable = true)
- `longitude`: double (nullable = true)
- `city_id`: long (nullable = true)
  - City ID, joinable to 
- `city`: string (nullable = true)
  - City full text
- `state`: string (nullable = true)
  - State two letter abbreviation

#### Step 5: Complete Project Write Up

* Clearly state the rationale for the choice of tools and technologies for the project.
  - Spark is a great choice (API simplicity, performance advantages) for analyzing and transforming large datasets that cannot fit into memory
  - SAS dataset has over 3M rows
  - Data is transformed and saved in S3 where anyone with appropriate permissions can access it
* Propose how often the data should be updated and why.
  - The immigration data represents one month of immigration transactions, so at minimum, should be updated monthly
  - Airports and City information are slower to change, so should be updated, perhaps, quarterly or yearly
  - More frequent updates would be necessary, due to user requirements
* Write a description of how you would approach the problem differently under the following scenarios:
  * The data was increased by 100x.
    - Potentially save datasets to AWS Redshift or EMR cluster for better performance and scalability
    - Potentially increase the number of Spark workers
    - Potentially partition the data in more ways, depending on querying habits
    - Potentially normalize the dataset even further to optimize storage
    - Potentially incorporate some OLAP cubes for faster BI analysis, depending on users' needs
  * The data populates a dashboard that must be updated on a daily basis by 7am every day.
    - Add an 'append' functionality that takes in new data, applies transformations, maps updates where appropriate, and data quality checks as needed to transform new and append to existing data
    - Build an Airflow DAG pipeline(s) that run at the appropriate time of night that apply the 'append' functionality to newly acquired data
      - Pipeline would have requisite checks that send alerts/emails if things fail
      - Incorporate logging for historical debugging
  * The database needed to be accessed by 100+ people.
    - Use AWS Redshift for readability performance and grant users credentials for access into the relevant part(s) of the dataset(s)
    - Create different views or aggregations to spread out the database and provide users a better experience